# Constructing drive time based service areas
This sample shows how the `network` module of the ArcGIS API for Python can be used to construct service areas. In this sample, we generate service areas for two of the fire stations in central Tokyo, Japan. We later observe how the service area varies by time of day for a fire station in the city of Los Angeles.

##### Connect to the GIS

In [56]:
from datetime import datetime
from IPython.display import HTML
import pandas as pd
from arcgis.gis import GIS
from getpass import getpass

user_name = 'arcgis_python'
password = getpass()
my_gis = GIS('https://www.arcgis.com', user_name, password)

········


In [89]:
my_gis._con.token

'dP7EUCxt4_GcM8CgNl7phooSvgkTR4_VXnwi63cDcsNGIMHiwqMHk1LjUoAxILq3lPDUKuGL7tRIrTHfFqwl6kCV-tkvLmdBLsVFGSdz6gm_szP83udGmtl_gkKMOCLO'

In [ ]:
from getpass import getpass

### Create a Network Layer

In [60]:
from arcgis.network import ServiceAreaLayer

In [61]:
service_area_url = my_gis.properties.helperServices.serviceArea.url
service_area_url

'https://route.arcgis.com/arcgis/rest/services/World/ServiceAreas/NAServer/ServiceArea_World'

In [62]:
sa_layer = ServiceAreaLayer(service_area_url, gis=my_gis)

## Calculate service area from SD convention center

In [63]:
from arcgis.geocoding import geocode
ps_geocode = geocode('Palm Springs Convention Center', as_featureset=True)
type(ps_geocode)

arcgis.features.feature.FeatureSet

Let us display the fire stations on a map

In [64]:
map1 = my_gis.map('Palm Springs, CA', zoomlevel=12)
map1

In [65]:
map1.draw(ps_geocode)

## Compute the service area


In [66]:
travel_modes = sa_layer.retrieve_travel_modes()

for t in travel_modes['supportedTravelModes']:
    print(t['name'])

Walking Time
Rural Driving Distance
Driving Time
Driving Distance
Walking Distance
Rural Driving Time
Trucking Time
Trucking Distance


In [67]:
truck_mode = [t for t in travel_modes['supportedTravelModes'] if t['name'] == 'Driving Time'][0]

In [88]:
result = sa_layer.solve_service_area(ps_geocode, default_breaks=[5,10,15], 
                                     travel_direction='esriNATravelDirectionToFacility',
                                     travel_mode=truck_mode)

#### Read the result back as a `FeatureSet`
The `result` variable contains the service area as a dictionary. We inspect its keys and construct `Feature` and `FeatureSet` objects out of it to display in the map

In [ ]:
result.keys()

In [ ]:
result['saPolygons'].keys()

In [69]:
poly_feat_list = []
for polygon_dict in result['saPolygons']['features']:
    f1 = Feature(polygon_dict['geometry'], polygon_dict['attributes'])
    poly_feat_list.append(f1)

In [70]:
service_area_fset = FeatureSet(poly_feat_list, 
                         geometry_type=result['saPolygons']['geometryType'],
                         spatial_reference= result['saPolygons']['spatialReference'])

### Visualize the service area on the map
From the DataFrame above, we know, there are 3 service area polygons for each fire station. The drive times are given as a range between `FromBreak` and `ToBreak` columns. Let us use this information to visualize the polygons with different colors and appropriate popup messags on the map

In [71]:
colors = {5: [0, 128, 0, 90], 
          10: [255, 255, 0, 90], 
          15: [255, 0, 0, 90]}

fill_symbol = {"type": "esriSFS","style": "esriSFSSolid",
               "color": [115,76,0,255]}

In [72]:
for service_area in service_area_fset.features:
    
    #set color based on drive time
    fill_symbol['color'] = colors[service_area.attributes['ToBreak']]
    
    #set popup
    popup={"title": "Service area", 
            "content": "{} minutes".format(service_area.attributes['ToBreak'])}
    
    map1.draw(service_area.geometry, symbol=fill_symbol, popup=popup)

Click the drive time areas to explore their attributes. Because the content of the pop-ups may include HTML source code, it is also possible to have the pop-up windows include other resources such as tables and images.

The service area has been computed, we process it to generate a list of `FeatureSet` objects to animate on the map

### Driving directions

In [73]:
from arcgis.network import RouteLayer

In [74]:
route_service_url = my_gis.properties.helperServices.route.url
route_service_url

'https://route.arcgis.com/arcgis/rest/services/World/Route/NAServer/Route_World'

In [75]:
route_service = RouteLayer(route_service_url, gis=my_gis)
route_service

<RouteLayer url:"https://route.arcgis.com/arcgis/rest/services/World/Route/NAServer/Route_World">

In [80]:
stops = ['Palm Springs Convention Center', 'Palm Springs Airport']
from arcgis.geocoding import geocode, batch_geocode
stops_geocoded = batch_geocode(stops)

#### data wrangling

In [81]:
stops_geocoded = [item['location'] for item in stops_geocoded]
stops_geocoded

[{'x': -116.53697007199997, 'y': 33.82585002600007},
 {'x': -116.50695999999999, 'y': 33.828070000000025}]

In [82]:
stops_geocoded2 = '{},{};{},{}'.format(stops_geocoded[0]['x'],stops_geocoded[0]['y'],
                                       stops_geocoded[1]['x'],stops_geocoded[1]['y'])
stops_geocoded2

'-116.53697007199997,33.82585002600007;-116.50695999999999,33.828070000000025'

### find dirving directions

In [83]:
result = route_service.solve(stops_geocoded2, 
                             return_routes=True,
                           return_stops=True, return_directions=True,
                           return_barriers=False, return_polygon_barriers=False,
                           return_polyline_barriers=False)

In [86]:
map2 = my_gis.map('Palm Springs, CA', zoomlevel=13)
map2

In [87]:
from arcgis.features import Feature, FeatureSet
features = result['routes']['features']
routes = FeatureSet(features)
stop_features = result['stops']['features']
stop_fset = FeatureSet(stop_features)
map2.draw(routes)
map2.draw(stop_fset)

In [ ]:
len(result['stops'])

In [ ]:
stop_features = result['stops']['features']
stop_fset = FeatureSet(stop_features)

In [ ]:
map2.draw(stop_fset)